The purpose of this notebook is to diagnose why I cannot straight-forwardly reproduce the MNL model used in "Brownstone, Davide and Train, Kenneth (1999). 'Forecasting new product penetration with flexible substitution patterns'. Journal of Econometrics 89: 109-129." (p. 121).

In [1]:
import sys
from collections import OrderedDict

import scipy.stats
import pandas as pd
import numpy as np
import pylogit as pl

sys.path.insert(0, '../src')
from visualization import predictive_viz as viz

%matplotlib inline

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
car_df = pd.read_csv("../data/interim/car_long_format.csv")

In [3]:
# Create the 'big_enough' variable
car_df['big_enough'] =\
    (car_df['hsg2'] & (car_df['vehicle_size'] == 3)).astype(int)

# Determine the type of car
car_df['sports_utility_vehicle'] =\
    (car_df['body_type'] == 'sportuv').astype(int)

car_df['sports_car'] =\
    (car_df['body_type'] == 'sportcar').astype(int)
    
car_df['station_wagon'] =\
    (car_df['body_type'] == 'stwagon').astype(int)

car_df['truck'] =\
    (car_df['body_type'] == 'truck').astype(int)

car_df['van'] =\
    (car_df['body_type'] == 'van').astype(int)

# Determine the car's fuel usage
car_df['electric'] =\
    (car_df['fuel_type'] == 'electric').astype(int)
    
car_df['non_electric'] =\
    (car_df['fuel_type'] != 'electric').astype(int)

car_df['compressed_natural_gas'] =\
    (car_df['fuel_type'] == 'cng').astype(int)

car_df['methanol'] =\
    (car_df['fuel_type'] == 'methanol').astype(int)

# Determine if this is an electric vehicle with a small commute
car_df['electric_commute_lte_5mi'] =\
    (car_df['electric'] & car_df['coml5']).astype(int)

# See if this is an electric vehicle for a college educated person
car_df['electric_and_college'] =\
    (car_df['electric'] & car_df['college']).astype(int)

# See if this is a methanol vehicle for a college educated person
car_df['methanol_and_college'] =\
    (car_df['methanol'] & car_df['college']).astype(int)
    
car_df['methanol_commute_lte_5mi'] =\
    (car_df['methanol'] & car_df['coml5']).astype(int)
    
# Scale the range and acceleration variables
car_df['range_over_100'] = car_df['range'] / 100.0
car_df['acceleration_over_10'] = car_df['acceleration'] / 10.0
car_df['top_speed_over_100'] = car_df['top_speed'] / 100.0
car_df['vehicle_size_over_10'] = car_df['vehicle_size'] / 10.0
car_df['tens_of_cents_per_mile'] = car_df['cents_per_mile'] / 10.0

In [4]:
car_mnl_spec, car_mnl_names = OrderedDict(), OrderedDict()

cols_and_display_names =\
    [('price_over_log_income', 'Price over log(income)'),
     ('range_over_100', 'Range (units: 100mi)'),
     ('acceleration_over_10', 'Acceleration (units: 0.1sec)'),
     ('top_speed_over_100', 'Top speed (units: 0.01mph)'),
     ('pollution', 'Pollution'),
     ('vehicle_size_over_10', 'Size'),
     ('big_enough', 'Big enough'),
     ('luggage_space', 'Luggage space'),
     ('tens_of_cents_per_mile', 'Operation cost'),
     ('station_availability', 'Station availability'),
     ('sports_utility_vehicle', 'Sports utility vehicle'),
     ('sports_car', 'Sports car'),
     ('station_wagon', 'Station wagon'),
     ('truck', 'Truck'),
     ('van', 'Van'),
     ('methanol', 'EV'),
     ('methanol_commute_lte_5mi', 'Commute < 5 & EV'),
     ('methanol_and_college', 'College & EV'),
     ('compressed_natural_gas', 'CNG'),
     ('electric', 'Methanol'),
     ('electric_and_college', 'College & Methanol')]
    
for col, display_name in cols_and_display_names:
    car_mnl_spec[col] = 'all_same'
    car_mnl_names[col] = display_name


In [5]:
# Initialize the mnl model
car_mnl = pl.create_choice_model(data=car_df,
                                 alt_id_col='alt_id',
                                 obs_id_col='obs_id',
                                 choice_col='choice',
                                 specification=car_mnl_spec,
                                 model_type='MNL',
                                 names=car_mnl_names)

# Create the initial variables for model estimation
num_vars = len(car_mnl_names)
initial_vals = np.zeros(num_vars)

# Estimate the mnl model
car_mnl.fit_mle(initial_vals, method='BFGS')

# Look at the estimation results
car_mnl.get_statsmodels_summary()

Log-likelihood at zero: -8,338.8486
Initial Log-likelihood: -8,338.8486


/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:385: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time: 0.21 seconds.
Final log-likelihood: -7,391.8300


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                4,654
Model:             Multinomial Logit Model   Df Residuals:                    4,633
Method:                                MLE   Df Model:                           21
Date:                     Sat, 09 Jun 2018   Pseudo R-squ.:                   0.114
Time:                             16:14:34   Pseudo R-bar-squ.:               0.111
converged:                           False   Log-Likelihood:             -7,391.830
                                             LL-Null:                    -8,338.849
================================================================================================
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Price over log(income)          -0.1854      0.027     -6.796      0.000        -0.239    -0.132
Range (units: 100mi)             0.3501      0.027     13.052      0.000         0.298     0.403
Acceleration (units: 0.1sec)    -0.7160      0.111     -6.472      0.000        -0.933    -0.499
Top speed (units: 0.01mph)       0.2612      0.081      3.228      0.001         0.103     0.420
Pollution                       -0.4441      0.102     -4.367      0.000        -0.643    -0.245
Size                             0.9345      0.316      2.953      0.003         0.314     1.555
Big enough                       0.1432      0.077      1.853      0.064        -0.008     0.295
Luggage space                    0.5009      0.191      2.623      0.009         0.127     0.875
Operation cost                  -0.7679      0.076    -10.131      0.000        -0.916    -0.619
Station availability             0.4133      0.096      4.294      0.000         0.225     0.602
Sports utility vehicle           0.8201      0.141      5.830      0.000         0.544     1.096
Sports car                       0.6370      0.148      4.298      0.000         0.347     0.928
Station wagon                   -1.4367      0.062    -23.139      0.000        -1.558    -1.315
Truck                           -1.0168      0.049    -20.753      0.000        -1.113    -0.921
Van                             -0.7989      0.047    -16.864      0.000        -0.892    -0.706
EV                              -0.1786      0.172     -1.041      0.298        -0.515     0.158
Commute < 5 & EV                 0.1983      0.084      2.374      0.018         0.035     0.362
College & EV                     0.4426      0.109      4.058      0.000         0.229     0.656
CNG                              0.3450      0.092      3.744      0.000         0.164     0.526
Methanol                         0.3134      0.103      3.051      0.002         0.112     0.515
College & Methanol               0.2284      0.089      2.576      0.010         0.055     0.402
================================================================================================
"""

Failed attempts:
1. Replace 'EV' with a non-EV dummy variable

Successful attempts:
1. Switch electric and methanol fuel type labels. The idea occurred to me because my original estimated methanol dummy had a similar coefficient to the estimated electric dummy variable in the paper.

# Wow. A Semantic Error?
Somehow the 'methanol' and 'electric' columns were erroneously swapped.

When I estimate the model, using 'methanol' as if it were 'electric' and vice-versa, I can recreate the model estimation results exactly.

# Check raw data from Brownstone and Train

There seems to have been an error going from their raw data to the data in mlogit. The dataset values don't match.

In [6]:
t_ = pd.read_table("../data/raw/mcfadden_train_2000_raw/mt-data/xmat.txt",
                   sep=r'\s*',
                   names=['col{}'.format(x) for x in range(1, 5)])

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [7]:
raw_df = pd.DataFrame(t_.values.reshape((4654, 156)))

In [8]:
# Look at the 'methanol' vehicles according to the original brownstone and train data
raw_df.iloc[0, 114:120]

114    0.0
115    0.0
116    1.0
117    1.0
118    0.0
119    0.0
Name: 0, dtype: float64

In [9]:
# Look at the fuel types for the first six long-format rows.
# From above, the third and fourth entry should be methanol vehicles
# NOT electric vehicles.
car_df['fuel_type'].iloc[:6]

0         cng
1         cng
2    electric
3    electric
4    gasoline
5    gasoline
Name: fuel_type, dtype: object